In [0]:
%sql
create database bronze

Importei diretamento para a camada bronze do meu computador uma planilha de veiculos eletricos nos estados unidos.

In [0]:
%sql 
SELECT * FROM bronze.electric_vehicle_population_data_8_csv LIMIT 10

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
KM8K33AGXL,King,Seattle,WA,98103,2020,HYUNDAI,KONA,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,258,0,43,249675142,POINT (-122.34301 47.659185),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033004800
1C4RJYB61N,King,Bothell,WA,98011,2022,JEEP,GRAND CHEROKEE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,1,233928502,POINT (-122.20578 47.762405),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033021804
1C4RJYD61P,Yakima,Yakima,WA,98908,2023,JEEP,GRAND CHEROKEE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,14,229675939,POINT (-120.6027202 46.5965625),PACIFICORP,53077002900
5YJ3E1EA7J,King,Kirkland,WA,98034,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,45,104714466,POINT (-122.209285 47.71124),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033021903
WBY7Z8C5XJ,Thurston,Olympia,WA,98501,2018,BMW,I3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,97,0,22,185498386,POINT (-122.89692 47.043535),PUGET SOUND ENERGY INC,53067010700
5YJ3E1EAXL,Snohomish,Marysville,WA,98271,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266,0,38,124595523,POINT (-122.1713847 48.10433),PUGET SOUND ENERGY INC,53061940001
2C4RC1N77H,King,Kent,WA,98042,2017,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,33,0,47,1815593,POINT (-122.111625 47.36078),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033031707
5YJYGDEE3L,King,Woodinville,WA,98072,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,45,124760555,POINT (-122.151665 47.75855),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033021906
5YJ3E1EA1J,Island,Coupeville,WA,98239,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,10,125048003,POINT (-122.6880708 48.2179983),PUGET SOUND ENERGY INC,53029971000
7SAYGDEF0P,King,Bellevue,WA,98004,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,0,0,48,240416207,POINT (-122.201905 47.61385),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033023806


In [0]:
%sql CREATE DATABASE silver

In [0]:
%sql 
    create table silver.Eletricvehicle stored as ORC as 
        (select County, City, State, `Model Year`,Make, Model, `Electric Vehicle Type`, `Electric Range`, `DOL Vehicle ID`  
         from bronze.electric_vehicle_population_data_8_csv
         Where County is not NULL and City is not NULL);



Fiz a transferencia da tabela para a camada Silver e fiz uma limpeza nas colunas Conty, City e Eletric Range

In [0]:
%sql 
SELECT * FROM silver.eletricvehicle LIMIT 10

County,City,State,Model Year,Make,Model,Electric Vehicle Type,Electric Range,DOL Vehicle ID
Pierce,Gig Harbor,WA,2016,TESLA,MODEL X,Battery Electric Vehicle (BEV),200,106365703
Chelan,Chelan,WA,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),210,176383504
King,Redmond,WA,2021,FORD,MUSTANG MACH-E,Battery Electric Vehicle (BEV),0,186478029
Snohomish,Snohomish,WA,2015,NISSAN,LEAF,Battery Electric Vehicle (BEV),84,189446995
Benton,Richland,WA,2019,HYUNDAI,IONIQ,Plug-in Hybrid Electric Vehicle (PHEV),29,221198932
Snohomish,Mukilteo,WA,2022,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),25,224674299
Mason,Belfair,WA,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),0,176611816
Spokane,Greenacres,WA,2022,TESLA,MODEL 3,Battery Electric Vehicle (BEV),0,209037321
King,Seattle,WA,2023,FORD,TRANSIT,Battery Electric Vehicle (BEV),0,227116235
King,Auburn,WA,2022,TESLA,MODEL Y,Battery Electric Vehicle (BEV),0,205524847


In [0]:
%sql CREATE DATABASE golden

In [0]:
%sql 
create table golden.eletricmedia stored as ORC as 
    select `Electric Vehicle Type`, AVG(`Electric Range`) as Mediadedistancia
    from silver.eletricvehicle
    group by `Electric Vehicle Type`

In [0]:
%sql
create table golden.QuantidadeXEstado stored as ORC as 
  SELECT
    `Electric Vehicle Type`,
    State,
    COUNT(*) AS QuantidadedeVeiculo
  FROM silver.eletricvehicle
  GROUP BY State,`Electric Vehicle Type`;

In [0]:
%sql
create table golden.TipoXMontadora stored as ORC as 
  SELECT
    `Electric Vehicle Type`,
    Make,
    COUNT(*) AS QuantidadedeVeiculo
    FROM silver.eletricvehicle
    GROUP BY Make,`Electric Vehicle Type`;

In [0]:
%sql
create table golden.QuantidadeXAno stored as ORC as 
  SELECT
    `Model Year`,
    COUNT(*) AS QuantidadedeVeiculo
  FROM silver.eletricvehicle
  GROUP BY `Model Year`;

In [0]:
%sql
create table golden.QuantidadeXMontadora stored as ORC as 
  SELECT
    `Make`,
    COUNT(*) AS quantidadedeveiculo
  FROM silver.eletricvehicle
  GROUP BY `Make`;

In [0]:
%sql
create table golden.eletricvehicle stored as ORC as 
  select *
  from silver.eletricvehicle


In [0]:
%sql 
SELECT * FROM golden.eletricvehicle LIMIT 10

County,City,State,Model Year,Make,Model,Electric Vehicle Type,Electric Range,DOL Vehicle ID
Pierce,Gig Harbor,WA,2016,TESLA,MODEL X,Battery Electric Vehicle (BEV),200,106365703
Chelan,Chelan,WA,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),210,176383504
King,Redmond,WA,2021,FORD,MUSTANG MACH-E,Battery Electric Vehicle (BEV),0,186478029
Snohomish,Snohomish,WA,2015,NISSAN,LEAF,Battery Electric Vehicle (BEV),84,189446995
Benton,Richland,WA,2019,HYUNDAI,IONIQ,Plug-in Hybrid Electric Vehicle (PHEV),29,221198932
Snohomish,Mukilteo,WA,2022,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),25,224674299
Mason,Belfair,WA,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),0,176611816
Spokane,Greenacres,WA,2022,TESLA,MODEL 3,Battery Electric Vehicle (BEV),0,209037321
King,Seattle,WA,2023,FORD,TRANSIT,Battery Electric Vehicle (BEV),0,227116235
King,Auburn,WA,2022,TESLA,MODEL Y,Battery Electric Vehicle (BEV),0,205524847


In [0]:
%sql 
SELECT * 
  FROM golden.tipoxmontadora LIMIT 10

Electric Vehicle Type,Make,QuantidadedeVeiculo
Battery Electric Vehicle (BEV),POLESTAR,764
Plug-in Hybrid Electric Vehicle (PHEV),CHRYSLER,2642
Battery Electric Vehicle (BEV),HYUNDAI,2695
Plug-in Hybrid Electric Vehicle (PHEV),TOYOTA,5034
Plug-in Hybrid Electric Vehicle (PHEV),ALFA ROMEO,12
Plug-in Hybrid Electric Vehicle (PHEV),CHEVROLET,4890
Battery Electric Vehicle (BEV),PORSCHE,547
Plug-in Hybrid Electric Vehicle (PHEV),FISKER,17
Battery Electric Vehicle (BEV),TH!NK,5
Plug-in Hybrid Electric Vehicle (PHEV),JEEP,3292
